# Road to Linearized Einstein Equations
The Linearized Einstein equation with eucledian background metric are of the form:
$$\left\lbrace
\begin{align*}
\text{div div S }\gamma &=0 \\
\text{div S }\left(\gamma' - 2 \epsilon \beta\right)  &=0 \\
\text{ S }\gamma'' +  \text{ inc }\gamma + \text{ S }\nabla \nabla \alpha - 2 \text{ S }\epsilon \beta ' &=0
\end{align*}
\right.
$$


$$


In [1]:
import time

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube
from ngsolve.krylovspace import CGSolver

## Approximation of the inc operator:
To approximate the inc operator we need to write te distributive form of it

In [16]:
n = specialcf.normal(3)
def norm(u, Mesh):
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh))
def Curl(u):
    if u.dim == 3:
        return CF( (u[1].Diff(z)- u[2].Diff(y), u[2].Diff(x)- u[0].Diff(z), u[0].Diff(y)- u[1].Diff(x)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )
def Inc(u):
    return Curl((Curl(u)).trans)
P_n = OuterProduct(n,n) 
Q_n = Id(3) - OuterProduct(n,n) 
C_n = CF(      (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) )
def C(V): 
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 



We need to imlement the following with $\sigma$ in $Reg^k$ and $\phi$ in lagrangian of order $k+1$ since the integrals over the Edges have to be well defined and the point evaluation is needed, therefore if $k=0$ then we need some continuity on the vertices!
$$
\sum_T \int_T inc (\sigma) \phi
\quad - \quad
2\sum_F \int_{F} C_n \, curl_f( u ) : \phi 
\quad - \quad
2\sum_F\int_{F} C_n \, rot_f( u ) : \phi 
\quad + \quad
\sum_E\int_{E} t^T( \phi C_n u  -u C_n \phi  ) n 
$$

With $$curl_f( u ) = curl(u Q_n)P_n $$ and  $$ rot_f( u ) = curl(u P_n)Q_n $$


In [13]:
def RunP(Maxh,Order):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    fesHCurlCurl = HCurlCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():
        ##########################################################
        #           Coefficient and grid functions               #
        ##########################################################
           
        # continuous functions
        peak = exp(-25*( (x-0.5)**2 + (y-0.5)**2 +(z-0.5)**2))

        PEAK = CF ( (peak, 0, 0 , 0,peak,0,0,0,peak), dims=(3,3))
        IncPeak = CF( Inc(PEAK), dims=(3,3) )

        # grid functions 
        gfG = GridFunction(fesHCurlCurl) 
        gfG.Set ( PEAK, bonus_intorder=10, dual=True)

        gfCurlCurlG = GridFunction(fesHCurlCurl)
        gfCurlCurlG.Set(IncPeak, BND,  bonus_intorder=9, dual=True)

        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################

        u,v = fesHCurlCurl.TnT()
        #some geometrical objects we need
        n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
        t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
        t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
        e = specialcf.tangential(3,True)
        n1 = Cross( t1, e)
        n2 = Cross( t2, e) 

        # Mass matrix
        a = BilinearForm(fesHCurlCurl, symmetric=True)
        a += InnerProduct(u,v)*dx 

        # linear form induced by the metric gfG
        f = LinearForm(fesHCurlCurl)

        # thetrahedron inc part
        f += InnerProduct(gfG.Operator("inc"),v)*dx        
        # faces part:
        f += ( InnerProduct(Q_n*n_cross_v, curl(gfG).trans) + Cross(gfG*n,n)*(curl(v)*n) )*dx(element_boundary=True)
        # Edges components: t'*v*C_n*n
        f += (gfG[n1,e]*v[e,t1] - gfG[n2,e]*v[e,t2])*dx(element_vb=BBND)
        
        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################
        pre = Preconditioner(a, "local")
        a.Assemble()
        f.Assemble()

        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfCurlCurlG.vec
        inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1000,tol=1e-9)
        gfCurlCurlG.vec.data += inverse * r
        #gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r

        print("error of my inc")
        Error = norm(IncPeak-gfCurlCurlG,mesh)
        print("Error inc(PEAK)-gfcurlCurlG when order is "+str(Order)+"-->"+str(Error) )
        Draw (gfCurlCurlG-IncPeak,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (gfCurlCurlG,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (IncPeak, mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 
        return Error

In [18]:
Maxh = 0.1
Order = 3
with TaskManager():
    RunP(Maxh, Order)

CG NOT converged in 1000 iterations to residual 1.3090458977620267e-05
error of my inc
Error inc(PEAK)-gfcurlCurlG when order is 3-->0.013797196072570373


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

## Cauchy Problem:
### Aproach number one: 
We define the cauchy problem: 
$$
\begin{align}
S \gamma '' + inc \gamma =0
\end{align}
$$
At each time step we calculate:
$$
\begin{align}
S \gamma_{n+1}- 2  S \gamma_{n} +  S \gamma_{n-1}  = - \tau^2 \text{ inc }\gamma_{n}
\end{align}
$$
$$
\begin{align}
S \gamma_{n+1} & = 2  S \gamma_{n} -  S \gamma_{n-1}- \tau^2 \text{ inc }\gamma_{n} \\
\gamma_{n+1} & = 2   \gamma_{n} -   \gamma_{n-1}- \tau^2S^{-1} \text{ inc }\gamma_{n} \\
\end{align}
$$




In [ ]:
def RunP(Maxh,Order):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    fesHCurlCurl = HCurlCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():
        ##########################################################
        #           Coefficient and grid functions               #
        ##########################################################
           
        # continuous functions
        peak = exp(-25*( (x-0.5)**2 + (y-0.5)**2 +(z-0.5)**2))

        PEAK = CF ( (peak, 0, 0 , 0,peak,0,0,0,peak), dims=(3,3))
        IncPeak = CF( Inc(PEAK), dims=(3,3) )

        # grid functions 
        gfG = GridFunction(fesHCurlCurl) 
        gfG.Set ( PEAK, bonus_intorder=10, dual=True)

        gfCurlCurlG = GridFunction(fesHCurlCurl)
        gfCurlCurlG.Set(IncPeak, BND,  bonus_intorder=9, dual=True)

        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################

        u,v = fesHCurlCurl.TnT()
        #some geometrical objects we need
        n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
        t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
        t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
        e = specialcf.tangential(3,True)
        n1 = Cross( t1, e)
        n2 = Cross( t2, e) 

        # Mass matrix
        a = BilinearForm(fesHCurlCurl, symmetric=True)
        a += InnerProduct(u,v)*dx 

        # linear form induced by the metric gfG
        f = LinearForm(fesHCurlCurl)

        # thetrahedron inc part
        f += InnerProduct(gfG.Operator("inc"),v)*dx        
        # faces part:
        f += ( InnerProduct(Q_n*n_cross_v, curl(gfG).trans) + Cross(gfG*n,n)*(curl(v)*n) )*dx(element_boundary=True)
        # Edges components: t'*v*C_n*n
        f += (gfG[n1,e]*v[e,t1] - gfG[n2,e]*v[e,t2])*dx(element_vb=BBND)
        
        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################
        pre = Preconditioner(a, "local")
        a.Assemble()
        f.Assemble()

        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfCurlCurlG.vec
        inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1000,tol=1e-9)
        gfCurlCurlG.vec.data += inverse * r
        #gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r

        print("error of my inc")
        Error = norm(IncPeak-gfCurlCurlG,mesh)
        print("Error inc(PEAK)-gfcurlCurlG when order is "+str(Order)+"-->"+str(Error) )
        Draw (gfCurlCurlG-IncPeak,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (gfCurlCurlG,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (IncPeak, mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 
        return Error

['ArnoldiSolver',
 'BBBND',
 'BBND',
 'BFI',
 'BND',
 'BSpline',
 'BVP',
 'BaseMatrix',
 'BaseVector',
 'BilinearForm',
 'BitArray',
 'BlockBFI',
 'BlockLFI',
 'BlockMatrix',
 'BlockVector',
 'BoundaryFromVolumeCF',
 'CELL',
 'CF',
 'CGSolver',
 'COUPLING_TYPE',
 'CacheCF',
 'CalcFlux',
 'CoefficientFunction',
 'Cof',
 'CompoundBFI',
 'CompoundLFI',
 'Compress',
 'CompressCompound',
 'Conj',
 'ConstEBEMatrix',
 'ContactBoundary',
 'ConvertOperator',
 'CreateVVector',
 'Cross',
 'Det',
 'Deviator',
 'DiagonalMatrix',
 'Discontinuous',
 'Draw',
 'DrawFlux',
 'EDGE',
 'ELEMENT',
 'ET',
 'ElementId',
 'Embedding',
 'FACE',
 'FACET',
 'FESpace',
 'FacetFESpace',
 'FacetSurface',
 'GMRESSolver',
 'Grad',
 'GridFunction',
 'H1',
 'H1LumpingFESpace',
 'HCurl',
 'HCurlCurl',
 'HCurlDiv',
 'HDiv',
 'HDivDiv',
 'HDivDivSurface',
 'HDivSurface',
 'HEX',
 'Hidden',
 'Id',
 'IdentityMatrix',
 'IfPos',
 'In',
 'InnerProduct',
 'IntRange',
 'Integrate',
 'IntegrationRule',
 'Interpolate',
 'Inv',
 'L2